In [ ]:
!nvidia-smi

Sun Oct 25 04:31:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import numpy as np
import pandas as pd
import torch

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

from tqdm import tqdm
tqdm.pandas()
import transformers
import ktrain
from ktrain import text

In [ ]:
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

In [ ]:
X_train = df_train['question_text']

In [ ]:
X_train = list(X_train)
X_train[0]

'Does provocative psychotherapy work ?'

In [ ]:
X_test = df_test['question_text']
X_test = list(X_test)
X_test[0]

'What is the best thing to do if your child experience bullying ?'

In [ ]:
y_train = df_train.target
y_train = list(y_train)

In [ ]:
y_test = df_test.target

y_test = list(y_test)

In [ ]:
MODEL_NAME = 'roberta-base'
t = text.Transformer(MODEL_NAME, maxlen=100, class_names=[0,1])
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=64)


preprocessing train...
language: en
train sequence lengths:
	mean : 17
	95percentile : 38
	99percentile : 50


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 17
	95percentile : 39
	99percentile : 50


In [ ]:
learner.fit_onecycle(1e-5, 5)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
2000/2000 [==============================] - 2620s 1s/step - loss: 0.2968 - accuracy: 0.8781 - val_loss: 0.2192 - val_accuracy: 0.9172
Epoch 2/5
2000/2000 [==============================] - 2621s 1s/step - loss: 0.2157 - accuracy: 0.9201 - val_loss: 0.2022 - val_accuracy: 0.9241
Epoch 3/5
2000/2000 [==============================] - 2616s 1s/step - loss: 0.1905 - accuracy: 0.9304 - val_loss: 0.1977 - val_accuracy: 0.9266
Epoch 4/5
2000/2000 [==============================] - 2617s 1s/step - loss: 0.1557 - accuracy: 0.9448 - val_loss: 0.2149 - val_accuracy: 0.9265
Epoch 5/5
2000/2000 [==============================] - 2605s 1s/step - loss: 0.1260 - accuracy: 0.9566 - val_loss: 0.2170 - val_accuracy: 0.9272


In [ ]:
learner.validate(class_names=[0,1])

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     15918
           1       0.93      0.93      0.93     16082

    accuracy                           0.93     32000
   macro avg       0.93      0.93      0.93     32000
weighted avg       0.93      0.93      0.93     32000



array([[14774,  1144],
       [ 1186, 14896]])